In [1]:
from PIL import Image, ImageDraw, ImageFilter, ImageFont
import numpy as np
import requests
from io import BytesIO
import requests
import pandas as pd
from dotenv import load_dotenv
import os
import pathlib


load_dotenv();

In [2]:

# obtener clave
api_key = os.getenv("GOOGLE_FONT_API_KEY")

url = f"https://www.googleapis.com/webfonts/v1/webfonts?key={api_key}"

# descargar JSON
response = requests.get(url)
raw_json = response.json()

df = pd.DataFrame(raw_json["items"])
df.head()


,family,variants,subsets,version,lastModified,files,category,kind,menu,colorCapabilities
0,ABeeZee,"[regular, italic]","[latin, latin-ext]",v23,2025-09-08,{'regular': 'https://fonts.gstatic.com/s/abeez...,sans-serif,webfonts#webfont,https://fonts.gstatic.com/s/abeezee/v23/esDR31...,NaN
1,ADLaM Display,[regular],"[adlam, latin, latin-ext]",v1,2025-05-30,{'regular': 'https://fonts.gstatic.com/s/adlam...,display,webfonts#webfont,https://fonts.gstatic.com/s/adlamdisplay/v1/KF...,NaN
2,AR One Sans,"[regular, 500, 600, 700]","[latin, latin-ext, vietnamese]",v6,2025-09-16,{'regular': 'https://fonts.gstatic.com/s/arone...,sans-serif,webfonts#webfont,https://fonts.gstatic.com/s/aronesans/v6/TUZez...,NaN
3,Abel,[regular],[latin],v18,2025-05-30,{'regular': 'https://fonts.gstatic.com/s/abel/...,sans-serif,webfonts#webfont,https://fonts.gstatic.com/s/abel/v18/MwQ5bhbm2...,NaN
4,Abhaya Libre,"[regular, 500, 600, 700, 800]","[latin, latin-ext, sinhala]",v18,2025-09-16,{'regular': 'https://fonts.gstatic.com/s/abhay...,serif,webfonts#webfont,https://fonts.gstatic.com/s/abhayalibre/v18/e3...,NaN


In [3]:
cols = ['family', 'variants', 'subsets', 'category', 'files']
df = df[cols]

df.head(5)

,family,variants,subsets,category,files
0,ABeeZee,"[regular, italic]","[latin, latin-ext]",sans-serif,{'regular': 'https://fonts.gstatic.com/s/abeez...
1,ADLaM Display,[regular],"[adlam, latin, latin-ext]",display,{'regular': 'https://fonts.gstatic.com/s/adlam...
2,AR One Sans,"[regular, 500, 600, 700]","[latin, latin-ext, vietnamese]",sans-serif,{'regular': 'https://fonts.gstatic.com/s/arone...
3,Abel,[regular],[latin],sans-serif,{'regular': 'https://fonts.gstatic.com/s/abel/...
4,Abhaya Libre,"[regular, 500, 600, 700, 800]","[latin, latin-ext, sinhala]",serif,{'regular': 'https://fonts.gstatic.com/s/abhay...


In [4]:
characters = ['1', '2', '3', '4', '5', '6', '7', '8', '9']
IMG_WIDTH = 100
IMG_HEIGHT = 100
count = 0


In [5]:
def drawCharacter(digit, font_url, font_size=50): 
    background_color = min(int(200 + np.random.normal() * 20),255)
    foreground_color = int(20 + np.random.normal() * 20)

    # Descargamos la fuente
    response = requests.get(font_url)
    font = ImageFont.truetype(BytesIO(response.content), font_size)
    
    # Imagen fondo blanco
    img = Image.new("L", (IMG_WIDTH, IMG_HEIGHT),(background_color,))
    draw = ImageDraw.Draw(img)

    # Tamaño real del caracter -> (left, top, right, bottom)

    bbox = draw.textbbox((0, 0), digit, font=font)

    text_w = bbox[2] - bbox[0]
    text_h = bbox[3] - bbox[1]

    # Dibujamos el numero
    draw.text(((IMG_WIDTH - text_w) // 2 - bbox[0], (IMG_HEIGHT - text_h) // 2 - bbox[1]), digit, fill= foreground_color, font=font)
    
    blurred = img.filter(filter=ImageFilter.GaussianBlur(1))

    left = IMG_WIDTH
    top = IMG_HEIGHT
    right = 0
    bottom = 0

    image_pixels = img.load()

    for x in range(IMG_WIDTH): 
        for y in range(IMG_HEIGHT): 
            if image_pixels[x, y] != background_color: 
                left = min(left, x)
                top = min(top, y)
                right = max(right, x)
                bottom = max(bottom, y)

    width = right - left + 4
    height = bottom - top + 4
    x = (left + right)/2
    y = (top + bottom)/2

    character_image = blurred.crop((x - width/2, y - height/2, x + width/2, y + height/2))

    # Agregamos ruido
    pixels = character_image.load()

    for x in range(character_image.size[0]):
        for y in range(character_image.size[1]):
            new_value = pixels[x,y] + np.random.normal() * 2
            pixels[x,y] = (int(new_value),)

    return character_image, np.array(img)


In [6]:
row = df[df.family == "ABeeZee"].iloc[0]
font_url = row["files"]["regular"]
font_url


'https://fonts.gstatic.com/s/abeezee/v23/esDR31xSG-6AGleN6tKukbcHCpE.ttf'

In [ ]:
for i in range(len(df)): 
    row = df.iloc[i]

    # Validar que exista "files" y que sea un diccionario
    files = row.get("files")

    # Validar que exista 'regular'
    if "regular" not in files:
        print(f"[Fila {i}] No tiene fuente 'regular'. Se omite.")
        continue

    font_url = files["regular"]

    # Validar que la URL no esté vacía
    if not font_url:
        print(f"[Fila {i}] 'regular' está vacío. Se omite.")
        continue

    family = row.get("family", "Unknown")

    # Procesar cada caracter
    for char in characters:
        try:
            path = f"testing_data/{char}" if count % 10 != 0 else f"training_data/{char}"
            pathlib.Path(path).mkdir(parents=True, exist_ok=True)

            img, arr = drawCharacter(char, font_url)

            img.save(f"{path}/{count}.png")
            count += 1

        except Exception as e:
            print(f"[Fila {i}, char '{char}'] Error procesando: {e}")
            continue



KeyError: 'regular'

In [7]:
img, arr = drawCharacter("2", font_url)
img


In [28]:
len(df)

1916

In [34]:
font_url

'https://fonts.gstatic.com/s/bubblerone/v22/f0Xy0eqj68ppQV9KBLmAouHH26MPePkt.ttf'